In [2]:
import numpy as np
from demand import sample_generation
import itertools
import pandas as pd
from tqdm import tqdm
from D_1_policy import dual_sourcing

In [ ]:
# 固定参数
c_r = 0
h = 1
T = 90
N = 1000
N_1 = 100

# 参数范围
c_e_list = [1, 2]
lt_pairs = [(3, 1), (5, 2), (10, 3)]
# l_r_list = [2, 3, 5, 10]
# l_e_list = [1, 2, 3]
service_level_list = [0.95, 0.975, 0.99]
# distributions = [("norm", (100, 10)), ("geom", (0.4)), ("binom", (100, 0.5))]
distributions = [("norm", (100, 10))]
results = []

# 遍历所有组合
for dist, c_e, lt_pair, service_level in \
    tqdm(itertools.product(distributions, c_e_list, lt_pairs, service_level_list)):
    l_r, l_e = lt_pair
    b = c_e + h * (l_r + 1)
    
    if b/(b+h)>=service_level:  # 跳过不符合条件的组合
        continue

    # if dist[0]=='norm':
    #     mean=100
    # elif dist[0]=='geom':
    #     mean=2.5
    # elif dist[0]=='binom':
    #     mean=50

    for seed_id in range(10):  # 10个随机种子测试稳定性
        
        random_seed1 = np.random.randint(10000)
        random_seed2 = np.random.randint(10000)
        print(seed_id, random_seed1, random_seed2)
        demand = sample_generation(dist, (N, T), random_seed=random_seed1)
        demand[demand < 0] = 0
        sample = sample_generation(dist, (N_1, 1000), random_seed=random_seed2)
        sample[sample < 0] = 0

        ds = dual_sourcing(c_r, c_e, h, b, l_r, l_e, demand, service_level)

        # 运行各策略
        try:
            single_result = ds.single_source(demand, S=None, inventory_level=0)
            print('single_result', single_result['average_total_cost'])
            ddi_result = ds.DDI_policy(demand, Se=None, D_2_constraint=False, inventory_level=0)
            print('ddi_result', ddi_result['average_total_cost'])
            tbs_result = ds.TBS_policy(sample, demand, x_init=None, q_init=None)
            print('tbs_result', tbs_result['average_total_cost'])
            benckmark_tbs_result = ds.cost_driven_TBS_policy(sample, demand, x_init=None, q_init=None)
            print('benckmark_tbs_result', benckmark_tbs_result['average_total_cost'])
            benckmark_di_result = ds.benchmark_DI_policy(demand, sample, x_init=None, q_init=None, inventory_level=0)
            print('benckmark_di_result', benckmark_di_result['average_total_cost'])
            di_result = ds.DI_policy(demand,sample,x_init=None,q_init=None,inventory_level=0)
            print('di_result', di_result['average_total_cost'])

            # 存结果
            results.append({
                "distribution": dist[0],
                "c_e": c_e,
                "l_r": l_r,
                "l_e": l_e,
                "service_level": service_level,
                "seed_id": seed_id,
                "b": b,
                "single_cost": single_result['average_total_cost'],
                "DDI_cost": ddi_result['average_total_cost'],
                "TBS_cost": tbs_result['average_total_cost'],
                "benckmark_TBS_result": benckmark_tbs_result['average_total_cost'],
                "benckmark_DI_result": benckmark_di_result['average_total_cost'],
                "DI_result": di_result['average_total_cost'],
                "random_seed1": random_seed1,
                "random_seed2": random_seed2,
            })
        except Exception as e:
            print(f"mistake in {dist, c_e, l_r, l_e, service_level, seed_id}: {e}")

# 保存结果到 CSV
df = pd.DataFrame(results)
df.to_csv(f"D1_result_{distributions[0][0]}.csv", index=False, encoding='utf-8-sig')
print("finished")

0it [00:00, ?it/s]

0 197 7166
single_result 3162.570301408636
ddi_result 3126.7623505359124
tbs_result 3153.073109828898
benckmark_tbs_result 3202.818721711699
benckmark_di_result 3154.3135713553997
最优的delta为229.4444009934264,对应的最优Se为203.469563145114
di_result 3152.9523026310912
1 8378 4913
single_result 3095.31576134028
ddi_result 3060.9758970457697


In [5]:
f"D1_result_{distributions[0][0]}"

'D1_result_norm'